In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tifffile import imread, imsave
import json

In [2]:
pathname = 'data/masks'
files = sorted(glob('data/masks/*.tif'))

labels = sorted(glob('data/labels/*.json'))
C = []
for el in labels:
    with open(el, 'r') as fp:
        C.append(json.load(fp))

In [3]:
masks = list(map(imread, files))
mask_labels = [np.unique(masks[i]) for i in range(len(masks))]
print(type(mask_labels[0]), len(mask_labels[0]))

<class 'numpy.ndarray'> 160


In [4]:
print(len(C))

6


In [5]:
# check if all labels in the json files are present also in the masks. in this case we know that some labels
# in the image are not present in the json. We eliminate them from the mask cause they were classified as NaN
for i in range(6):
    print("Processing file ", files[i])
    ids_labels = set(C[i].keys())
    ids_mask = set(str(el) for el in mask_labels[i])
    
    ids_labels = set(ids_labels)
    ids_mask = set(ids_mask)
    print(ids_labels - ids_mask)
    print(ids_mask - ids_labels)
    labels_to_eliminate_from_mask = ids_mask - ids_labels
    labels_to_eliminate_from_json = ids_labels - ids_mask
    
    if len(labels_to_eliminate_from_mask) > 0:
        for el in labels_to_eliminate_from_mask:
            masks[i] = np.where(masks[i] ==int(el), 0, masks[i])
            
    if len(labels_to_eliminate_from_json) > 0:
        print(len(C[i]))
        for el in labels_to_eliminate_from_json:
            C[i].pop(el) 
        print(len(C[i]))
        with open('data/labels/checked/'+os.path.basename(files[i])[:-3]+'json', 'w') as fp:
            json_ids = json.dump(C[i], fp)
        
    
    
    filename = pathname+'/checked/'+os.path.basename(files[i])
    imsave(filename, masks[i])
    
  

Processing file  data/masks/fish2_nasal_15-75z.tif
set()
set()
Processing file  data/masks/fish2_nasal_20-80z.tif
{'2499', '2277'}
set()
227
225
Processing file  data/masks/fish2_nasal_25-85z.tif
{'2028', '1485', '2599', '1667', '3031', '2893', '2488', '3584'}
set()
285
277
Processing file  data/masks/fish2_temporal_10-70z.tif
{'3093', '3026', '3386', '4970', '4439', '3832', '3206', '507', '4967', '4011', '95', '1413', '4553', '5374', '3990', '2515', '4060', '1302', '4109'}
set()
317
298
Processing file  data/masks/fish2_temporal_20-81z.tif
{'5587', '5558'}
set()
171
169
Processing file  data/masks/fish2_temporal_5-65z.tif
set()
set()


In [6]:
# check if new masks correspond to 
new_masks = sorted(glob(pathname+'/checked/*.tif'))
new_masks = list(map(imread, new_masks))
new_mask_labels = [np.unique(new_masks[i]) for i in range(len(new_masks))]

new_labels = sorted(glob('data/labels/checked/*.json'))
CC = []
for el in new_labels:
    with open(el, 'r') as fp:
        CC.append(json.load(fp))

for i in range(6):
    print("Processing file ", files[i])
    ids_new_labels = set(CC[i].keys())
    ids_new_mask = set(str(el) for el in new_mask_labels[i])
    
    ids_new_labels = set(ids_new_labels)
    ids_new_mask = set(ids_new_mask)
    print(ids_new_labels - ids_new_mask)
    print(ids_new_mask - ids_new_labels)
   

Processing file  data/masks/fish2_nasal_15-75z.tif
set()
set()
Processing file  data/masks/fish2_nasal_20-80z.tif
set()
set()
Processing file  data/masks/fish2_nasal_25-85z.tif
set()
set()
Processing file  data/masks/fish2_temporal_10-70z.tif
set()
set()
Processing file  data/masks/fish2_temporal_20-81z.tif
set()
set()
Processing file  data/masks/fish2_temporal_5-65z.tif
set()
set()


In [12]:
a = sorted(np.unique(masks[2]))[1:]
b = list(C[2].keys())

for i in range(len(a)):
    if a[i] != b[i]:
        print(a[i], b[i], type(a[i]), type(b[i]) )

9 9 <class 'numpy.uint32'> <class 'str'>
72 72 <class 'numpy.uint32'> <class 'str'>
79 79 <class 'numpy.uint32'> <class 'str'>
115 115 <class 'numpy.uint32'> <class 'str'>
154 154 <class 'numpy.uint32'> <class 'str'>
178 178 <class 'numpy.uint32'> <class 'str'>
197 197 <class 'numpy.uint32'> <class 'str'>
198 198 <class 'numpy.uint32'> <class 'str'>
220 220 <class 'numpy.uint32'> <class 'str'>
230 230 <class 'numpy.uint32'> <class 'str'>
254 254 <class 'numpy.uint32'> <class 'str'>
287 287 <class 'numpy.uint32'> <class 'str'>
310 310 <class 'numpy.uint32'> <class 'str'>
326 326 <class 'numpy.uint32'> <class 'str'>
343 343 <class 'numpy.uint32'> <class 'str'>
369 369 <class 'numpy.uint32'> <class 'str'>
394 394 <class 'numpy.uint32'> <class 'str'>
404 404 <class 'numpy.uint32'> <class 'str'>
434 434 <class 'numpy.uint32'> <class 'str'>
441 441 <class 'numpy.uint32'> <class 'str'>
473 473 <class 'numpy.uint32'> <class 'str'>
474 474 <class 'numpy.uint32'> <class 'str'>
478 478 <class 'nu

In [ ]:
classes = {'RGC': 1, 'BC': 2, 'PR': 3, 'AC': 4, 'HC': 5, 'LENS': 6, 'RPE': 7, np.nan : 8}

converted_classes = df['class'].map(classes)
print(converted_classes)

In [ ]:
ids = {k:v for k,v in zip(df['label'], converted_classes)}
ids[0] = 0
print(ids)

In [ ]:
print(len(df), len(ids))

In [ ]:
mask = masks[0]
m = mask.flatten()
print(m)
new_mask = []
for i in range(len(m)):
    if m[i] not in ids:
        new_mask.append(8)
    else:
        new_mask.append(ids[m[i]])
    
print(new_mask)

In [ ]:
print(masks[0].shape)

new_mask = np.asarray(new_mask)
new_mask = np.reshape(new_mask, masks[0].shape)
print(new_mask.shape)

In [ ]:
plt.imshow(new_mask[10])
plt.show()

In [ ]:
import napari

viewer = napari.view_image(new_mask)


In [ ]:
viewer = napari.view_labels(new_mask, scale=(2, 1, 1))

In [ ]:
# sava data as json file
import json
with open(path_mask+'/labels/'+os.path.basename(labels[0])[:-3]+'json', 'w') as fp:
    json_ids = json.dump(ids, fp)
    

In [ ]:
!ls


In [ ]:
!cd masks


In [ ]:
path_mask

In [ ]:
!ls /Users/lucrezia/Desktop/StarDistData/ELISA/project9_classification_of_neuronal_types/masks/complete_masks/manually_corrected/labels/